In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

In [3]:
%run -i data_generation/simple_linear_regression.py


In [4]:
%run -i data_preparation/v0.py


In [5]:
%run -i model_configuration/v0.py

In [6]:
# %load model_training/v0.py
# Defines number of epochs
n_epochs = 1000

for epoch in range(n_epochs):
    # Set model to TRAIN mode
    model.train()
    # Step 1- Computes model's predicted output - forward parss
    yhat = model(x_train_tensor)
    # Step 2 = Computes the loss
    loss = loss_fn(yhat, y_train_tensor)
    # Step 3 - Computes gradients for both "b" and "w" paramters
    loss.backward()
    # Step 4 - Updats parameters using gradients and the learning rate
    optimizer.step()
    optimizer.zero_grad()

In [7]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9690]], device='cuda:0')), ('0.bias', tensor([1.0235], device='cuda:0'))])


In [8]:
def make_train_step_fn(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def perform_train_step_fn(x, y):
        # Set model to TRAIN mode
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return perform_train_step_fn

In [9]:
%run -i data_preparation/v0.py

In [10]:
%%writefile model_configuration/v1.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'

lr = 0.1
torch.manual_seed(42)
model = nn.Sequential(nn.Linear(1,1)).to(device)

optimizer = optim.SGD(model.parameters(), lr=lr)

loss_fn = nn.MSELoss(reduction='mean')

train_step_fn = make_train_step_fn(model, loss_fn, optimizer)

Overwriting model_configuration/v1.py


In [11]:
%run -i model_configuration/v1.py

In [12]:
train_step_fn

<function __main__.make_train_step_fn.<locals>.perform_train_step_fn(x, y)>

In [13]:
%%writefile model_training/v1.py

n_epochs = 1000
losses = []

for epoch in range(n_epochs):
    loss = train_step_fn(x_train_tensor, y_train_tensor)
    losses.append(loss)

Overwriting model_training/v1.py


In [14]:
%run -i model_training/v1.py

In [15]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9690]], device='cuda:0')), ('0.bias', tensor([1.0235], device='cuda:0'))])


In [16]:
class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    def __len__(self):
        return len(self.x)

x_train_tensor = torch.as_tensor(x_train).float()
y_train_tensor = torch.as_tensor(y_train).float()

train_data = CustomDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([0.7713]), tensor([2.4745]))


In [17]:
tain_data = TensorDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([0.7713]), tensor([2.4745]))


In [18]:
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True,)

In [19]:
next(iter(train_loader))

[tensor([[0.2809],
         [0.3253],
         [0.1560],
         [0.5924],
         [0.0651],
         [0.8872],
         [0.4938],
         [0.0055],
         [0.1409],
         [0.0885],
         [0.1849],
         [0.7290],
         [0.8662],
         [0.3117],
         [0.6842],
         [0.1987]]),
 tensor([[1.5846],
         [1.8057],
         [1.2901],
         [2.1687],
         [1.1559],
         [2.8708],
         [1.9060],
         [1.0632],
         [1.1211],
         [1.0708],
         [1.5888],
         [2.4927],
         [2.6805],
         [1.7637],
         [2.3492],
         [1.2654]])]

In [20]:
%%writefile data_preparation/v1.py

x_train_tensor = torch.as_tensor(x_train).float()
y_train_tensor = torch.as_tensor(y_train).float()

# Builds Dataset
train_data = TensorDataset(x_train_tensor, y_train_tensor)

# Builds DataLoader
train_loader = DataLoader(
    dataset=train_data,
    batch_size=16,
    shuffle=True,
)

Overwriting data_preparation/v1.py


In [21]:
%run -i data_preparation/v1.py

In [22]:
%run -i model_configuration/v1.py

In [23]:
%%writefile model_training/v2.py

# Defines number of epochs
n_epochs = 1000

losses = []

# For each epoch...
for epoch in range(n_epochs):
    # inner loop
    mini_batch_losses = []
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        # Performs one train step and returns the 
        # corresponding loss for this mini-batch
        mini_batch_loss = train_step_fn(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)
    # Computes average loss over all mini-batches
    # That's the epoch loss
    loss = np.mean(mini_batch_loss)
    losses.append(loss)

Overwriting model_training/v2.py


In [24]:
%run -i model_training/v2.py

In [25]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9696]], device='cuda:0')), ('0.bias', tensor([1.0243], device='cuda:0'))])


In [30]:
def mini_batch(device, data_loader, step_fn):
    mini_batch_losses = []
    for x_batch, y_batch in data_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        mini_batch_loss = step_fn(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)
    loss = np.mean(mini_batch_losses)
    return loss

In [31]:
%run -i data_preparation/v1.py
%run -i model_configuration/v1.py

In [32]:
%%writefile model_training/v3.py

# Defines number of epochs
n_epochs = 200

losses = []

for epoch in range(n_epochs):
    # innner loop
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)

Overwriting model_training/v3.py


In [33]:
%run -i model_training/v3.py

In [34]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9684]], device='cuda:0')), ('0.bias', tensor([1.0219], device='cuda:0'))])


In [35]:
%%writefile data_preparation/v2.py

torch.manual_seed(13)
# Builds tensors from numpy arrays BEFORE  split
x_tensor = torch.as_tensor(x).float()
y_tensor = torch.as_tensor(y).float()

# Builds dataset containing ALL data points
dataset = TensorDataset(x_tensor, y_tensor)

# Performas the split
ratio = .8
n_total = len(dataset)
n_train = int(n_total * ratio)
n_val = n_total - n_train
train_data, val_data = random_split(dataset, [n_train,n_val])

# Buidls a loader of each set
train_loader = DataLoader(
    dataset=train_data,
    batch_size=16,
    shuffle=True,
)
val_loader = DataLoader(dataset=val_data, batch_size=16)

Writing data_preparation/v2.py


In [36]:
%run -i data_preparation/v2.py

In [37]:
def make_val_step_fn(model, loss_fn):
    # Builds function that performs a step in
    # the validation loop
    def perform_val_step_fn(x, y):
        model.eval()
        # step 1 - Computes our model's predicted output
        # forward pass
        yhat = model(x)
        # Step 2 - computes the losss
        loss = loss_fn(yhat, y)
        # There is not need step3 and 4, 
        # since we don't update parameters during evaluation
        return loss.item()
    return perform_val_step_fn

In [40]:
%%writefile model_configuration/v2.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'

lr = 0.1

torch.manual_seed(42)
model = nn.Sequential(nn.Linear(1,1)).to(device)
# Define an SGD optimizer to update the parameters
optimizer = optim.SGD(model.parameters(), lr=lr)

# Define an MSE loss function
loss_fn = nn.MSELoss(reduction='mean')

# Create the train_step function for our model, loss function and optimizer
train_step_fn = make_train_step_fn(model, loss_fn, optimizer)

# Create the val_step function for our model and loss function
val_step_fn = make_val_step_fn(model, loss_fn)

Overwriting model_configuration/v2.py


In [41]:
%run -i model_configuration/v2.py

In [42]:
%%writefile model_training/v4.py

n_epochs = 200
losses = []
val_losses = []

for epoch in range(n_epochs):
    # inner loop
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)

    # Validation -no gradients in validation!
    with torch.no_grad():
        val_loss =mini_batch(device, val_loader, val_step_fn)
        val_losses.append(val_loss)

Writing model_training/v4.py


In [43]:
%run -i model_training/v4.py

In [44]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9438]], device='cuda:0')), ('0.bias', tensor([1.0287], device='cuda:0'))])


In [45]:
%load_ext tensorboard

In [53]:
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 20196), started 0:09:13 ago. (Use '!kill 20196' to kill it.)

In [47]:
writer = SummaryWriter('runs/test')

In [49]:
dummy_x, dummy_y = next(iter(train_loader))

writer.add_graph(model, dummy_x.to(device))

In [52]:
writer.add_scalars(
    main_tag='loss',
    tag_scalar_dict={'training': loss, 'validation': val_loss},
    global_step=epoch
)

In [54]:
%run -i data_preparation/v2.py

In [55]:
%%writefile model_configuration/v3.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'

lr = 0.1

torch.manual_seed(42)
model = nn.Sequential(nn.Linear(1,1)).to(device)
# Define an SGD optimizer to update the parameters
optimizer = optim.SGD(model.parameters(), lr=lr)

# Define an MSE loss function
loss_fn = nn.MSELoss(reduction='mean')

# Create the train_step function for our model, loss function and optimizer
train_step_fn = make_train_step_fn(model, loss_fn, optimizer)

# Create the val_step function for our model and loss function
val_step_fn = make_val_step_fn(model, loss_fn)

# Creates a Summary Writer to itnerface with TensorBoard
writer = SummaryWriter('runs/simple_linear_regression')
dummy_x, dummy_y = next(iter(train_loader))

writer.add_graph(model, dummy_x.to(device))

Writing model_configuration/v3.py


In [56]:
%run -i model_configuration/v3.py

In [57]:
%%writefile model_training/v5.py

n_epochs = 200
losses = []
val_losses = []

for epoch in range(n_epochs):
    # inner loop
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)

    # Validation -no gradients in validation!
    with torch.no_grad():
        val_loss =mini_batch(device, val_loader, val_step_fn)
        val_losses.append(val_loss)
    # Records both losses for each epoch under tag "loss"
    writer.add_scalars(main_tag='loss',
                      tag_scalar_dict={
                          'training': loss,
                          'validation': val_loss},
                       global_step=epoch)

writer.close()

Writing model_training/v5.py


In [58]:
%run -i model_training/v5.py

In [59]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9432]], device='cuda:0')), ('0.bias', tensor([1.0263], device='cuda:0'))])


In [61]:
%tensorboard --logdir runs/simple_linear_regression

In [65]:
checkoutpoint = {'epoch': n_epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': losses,
                'val_loss': val_losses}
torch.save(checkoutpoint, 'model_checkpoint.pth')

In [66]:
%run -i data_preparation/v2.py
%run -i model_configuration//v3.py

In [67]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[0.7645]], device='cuda:0')), ('0.bias', tensor([0.8300], device='cuda:0'))])


In [68]:
checkpoint = torch.load('model_checkpoint.pth')

model.load_state_dict(checkoutpoint['model_state_dict'])
optimizer.load_state_dict(checkoutpoint['optimizer_state_dict'])

saved_epoch = checkoutpoint['epoch']
saved_losses = checkoutpoint['loss']
saved_val_losses = checkoutpoint['val_loss']

model.train()  # always use TRAIN fro resuming traning

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
)

In [69]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9432]], device='cuda:0')), ('0.bias', tensor([1.0263], device='cuda:0'))])


In [70]:
%run -i model_training/v5.py

In [71]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9432]], device='cuda:0')), ('0.bias', tensor([1.0263], device='cuda:0'))])


In [72]:
%run -i model_configuration/v3.py

In [73]:
checkpoint = torch.load('model_checkpoint.pth')

model.load_state_dict(checkoutpoint['model_state_dict'])
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9432]], device='cuda:0')), ('0.bias', tensor([1.0263], device='cuda:0'))])


In [74]:
new_inputs = torch.tensor([[.20], [.34], [.57]])

model.eval() # always use EVAL for fullly trained models
model(new_inputs.to(device))

tensor([[1.4150],
        [1.6870],
        [2.1340]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [76]:
# %load data_preparation/v2.py

torch.manual_seed(13)
# Builds tensors from numpy arrays BEFORE  split
x_tensor = torch.as_tensor(x).float()
y_tensor = torch.as_tensor(y).float()

# Builds dataset containing ALL data points
dataset = TensorDataset(x_tensor, y_tensor)

# Performas the split
ratio = .8
n_total = len(dataset)
n_train = int(n_total * ratio)
n_val = n_total - n_train
train_data, val_data = random_split(dataset, [n_train,n_val])

# Buidls a loader of each set
train_loader = DataLoader(
    dataset=train_data,
    batch_size=16,
    shuffle=True,
)
val_loader = DataLoader(dataset=val_data, batch_size=16)


In [77]:
# %load model_configuration/v3.py

device = 'cuda' if torch.cuda.is_available() else 'cpu'

lr = 0.1

torch.manual_seed(42)
model = nn.Sequential(nn.Linear(1,1)).to(device)
# Define an SGD optimizer to update the parameters
optimizer = optim.SGD(model.parameters(), lr=lr)

# Define an MSE loss function
loss_fn = nn.MSELoss(reduction='mean')

# Create the train_step function for our model, loss function and optimizer
train_step_fn = make_train_step_fn(model, loss_fn, optimizer)

# Create the val_step function for our model and loss function
val_step_fn = make_val_step_fn(model, loss_fn)

# Creates a Summary Writer to itnerface with TensorBoard
writer = SummaryWriter('runs/simple_linear_regression')
dummy_x, dummy_y = next(iter(train_loader))

writer.add_graph(model, dummy_x.to(device))


In [78]:
# %load model_training/v5.py

n_epochs = 200
losses = []
val_losses = []

for epoch in range(n_epochs):
    # inner loop
    loss = mini_batch(device, train_loader, train_step_fn)
    losses.append(loss)

    # Validation -no gradients in validation!
    with torch.no_grad():
        val_loss =mini_batch(device, val_loader, val_step_fn)
        val_losses.append(val_loss)
    # Records both losses for each epoch under tag "loss"
    writer.add_scalars(main_tag='loss',
                      tag_scalar_dict={
                          'training': loss,
                          'validation': val_loss},
                       global_step=epoch)

writer.close()


In [79]:
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9432]], device='cuda:0')), ('0.bias', tensor([1.0263], device='cuda:0'))])
